In [1]:
from src.v4.torchdata import load_vals, unload_vals, print_formatted_table, perturb
from src.v5.problem import symbolic, acos, wrap_sympy_function
from src.v6.problem import MFunctionalSet, MFunctionalSetLeaf, get_sharedvars
from scipy import optimize
from sympy.utilities.lambdify import implemented_function
import sympy as sp
import numpy as np # for np.pi
import torch

In [2]:
def torch_linear_interp1d(x_new, x, y):
    # Assumes x is 1D, sorted, and no duplicate values
    idxs = torch.searchsorted(x, x_new, right=True)
    idxs = torch.clamp(idxs, 1, x.size(0)-1)
    x0, x1 = x[idxs-1], x[idxs]
    y0, y1 = y[idxs-1], y[idxs]
    weight = (x_new - x0) / (x1 - x0)
    return y0 + weight * (y1 - y0)

In [3]:
h_i = torch.tensor([100, 150, 200, 250, 300, 350, 400, 450, 500, 
                    550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 1250, 1500]) * 1e3
rho_i = torch.tensor([4.79e-07, 1.81e-09, 2.53e-10, 6.24e-11, 1.95e-11, 6.98e-12, 2.72e-12, 1.13e-12, 4.89e-13, 2.21e-13, 1.04e-13, 5.15e-14, 2.72e-14, 1.55e-14, 9.63e-15, 6.47e-15, 4.66e-15, 3.54e-15, 2.79e-15, 1.11e-15, 5.21e-16])
H_i = torch.tensor([5.9, 25.5, 37.5, 44.8, 50.3, 54.8, 58.2, 61.3, 64.5, 68.7, 74.8, 84.4, 99.3, 121, 151, 188, 226, 263, 296, 408, 516]) * 1e3

rhointerp = lambda x: torch_linear_interp1d(x, torch.log(h_i), torch.log(rho_i))
Hinterp = lambda x: torch_linear_interp1d(x, torch.log(h_i), torch.log(H_i))
rhoapprox = lambda h: torch.tensor([rho_i[0]], dtype=torch.float64) if h<=h_i[0] else torch.exp(rhointerp(torch.log(h))) if h <= 1500e3 else torch.tensor([5.21e-16], dtype=torch.float64)
Happrox = lambda h: torch.tensor([H_i[0]], dtype=torch.float64) if h<=h_i[0] else torch.exp(Hinterp(torch.log(h))) if h <= 1500e3 else torch.tensor([516e3], dtype=torch.float64)


In [4]:
h, a, T, g, d, r = symbolic('h', 'a', 'T', 'g', 'd', 'r')
R, mu = 6378e3, 3.986e14
Orbit = MFunctionalSetLeaf(
    a==h+R, 
    T==2*np.pi*(a**3/mu)**0.5, 
    g==1/np.pi*acos(R/a), 
    d==g+0.5, 
    r==(h**2+2*R*h)**0.5
) # by default configured to elimination of state variables

m_A, P_c, P_T, E_b, m_b, A = symbolic('m_A', 'P_c', 'P_T', 'E_b', 'm_b', 'A')
eta_A, rho_A, rho_b, P_l, Q = 0.3, 10, 0.002e-3, 12, 1367
Power = MFunctionalSetLeaf(
    m_A==rho_A*A, 
    P_c==d*A*Q*eta_A, 
    P_T==P_c-P_l, 
    E_b==P_c*T/d, 
    m_b==rho_b*E_b
)

X_r, D_p, D, m_p = symbolic('X_r', 'D_p', 'D', 'm_p')
rho_p, l_v, B, N = 2, 500e-9, 8, 2000
Payload = MFunctionalSetLeaf(
    D_p==1.22*l_v*h/X_r, 
    D==2*np.pi*R*B*N/X_r, 
    m_p==rho_p*D_p**1.5
)

b, l_c, D_T, m_T, EN, G_r = symbolic('b', 'l_c', 'D_T', 'm_T', 'EN', 'G_r')
dBtoLinear = lambda db: 10**(db/10)
rho_T, G_T, D_r, eta, T_s, k, c, F = 0.2, dBtoLinear(16.5), 5.3, 0.55, 135, 1.38064852e-23, 2.99e8, 2.2e9
L = dBtoLinear(1+8.5+0.3+0.1)
Comms = MFunctionalSetLeaf(
    b==D/T, 
    l_c==c/F,
    D_T==l_c*(G_T/eta)**0.5/np.pi,
    m_T==rho_T*D_T**1.5,
    G_r==eta*(np.pi*D_r/l_c)**2,
    EN==P_T*G_r*G_T/(L*k*T_s*b)*(l_c/(4*np.pi*r))**2,
)

mt, m_s, m_pr = symbolic('m_t', 'm_s', 'm_pr')
eta_S = 0.2
# m_pr = 0.5
Struct = MFunctionalSetLeaf(
    m_s == eta_S*mt
)
Mass = MFunctionalSetLeaf(
    mt == m_A + m_b + m_p + m_T + m_pr + m_s 
)

Ln, Lp, Lt = symbolic('Ln', 'Lp', 'Lt')
L_min, C_D, I_sp, G = 10, 2.2, 70, 9.81
H = wrap_sympy_function(implemented_function(sp.Function('H'), Happrox))
rho = wrap_sympy_function(implemented_function(sp.Function('rho'),  rhoapprox))

Propulsion = MFunctionalSetLeaf(
    Ln == H(h)*mt/(2*np.pi*C_D*A*rho(h)*a**2)*T/31536000,
    Lp == m_pr*I_sp*G*a/(0.5*C_D*A*H(h)*mu)/31536000,
    Lt == Ln + Lp
)

MDA = MFunctionalSet(Orbit, Power, Payload, Comms, Propulsion, Struct, Mass)
FPF = MDA.subsetof( 1 <= X_r, X_r <= 5, 0<=m_pr,
                   400e3 >= h, 10 <= Lt, 14.125<=EN).minimize(mt)

In [5]:
SPF_MDF = FPF.config(elim=[Orbit, Power, Payload, Comms, MFunctionalSet(Struct, Mass).config(residuals=[Struct, Mass]), Propulsion])
SPF_IDF = FPF.config(parallel=FPF.supersets)
SPF_AAO = FPF.config(residuals=FPF.supersets) 

In [8]:
_, ineq, eq, obj, indices = SPF_MDF.gather_sets()
sharedidxs = get_sharedvars([elt.build(indices=indices).analysis for elt in MDA.supersets]+[ineq]+eq+[obj]) # need to include constraints and objectives
sharedvars = [key for key,var in indices.items() if var.item() in sharedidxs]
sharedvars, len(sharedvars), len(indices)

([Lt, EN, m_t, m_s, m_T, m_A, m_b, m_p, a, T, r, P_T, D, d], 14, 28)

### MDF

In [10]:
f_MDF = SPF_MDF.build()
x0 = {"h": 500e3, "m_pr": 1, "X_r": 10, "A":0.06}
x0_MDA = f_MDF.analysis(load_vals(x0, f_MDF.indices, isdict=True, default=1.1))
for elt in SPF_MDF.supersets:
    fP = elt.build()
    xP = load_vals(unload_vals(x0_MDA, f_MDF.indices), fP.indices, isdict=True)
    print_formatted_table([xP], fP.indices)

r      h      d     g     a      T       
2.57e6 5.00e5 0.622 0.122 6.88e6 5676.812
m_b   E_b    d     P_c    T        P_T   A    m_A
0.279 1.40e5 0.622 15.308 5676.812 3.308 0.06 0.6
m_p   D_p  D        X_r h     
0.011 0.03 6.41e+10 10  5.00e5
EN      b      G_r      l_c   r      P_T   m_T   D_T  D        T       
104.632 1.13e7 8255.002 0.136 2.57e6 3.308 0.049 0.39 6.41e+10 5676.812
Lt    Lp Ln    A    h      m_pr a      m_t   T       
1.467 0  1.467 0.06 5.00e5 1    6.88e6 2.423 5676.812
m_s   m_t  
0.485 2.423
m_t   m_s   m_T   m_pr m_A m_b   m_p  
2.423 0.485 0.049 1    0.6 0.279 0.011


In [23]:
obj, dobj, xguess, cons, idxs, solidxs = SPF_MDF.build_opt(x0={str(key): xguess2[val].item() for key,val in indices.items()})

In [24]:
xsol = optimize.minimize(obj, xguess, jac=dobj, 
                         constraints=cons, method='SLSQP')
xsol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.804200911257506
       x: [ 5.087e-02  4.000e+02  3.833e+00  5.000e+00]
     nit: 7
     jac: [ 1.819e+01  1.652e-04  1.250e+00 -8.085e-03]
    nfev: 8
    njev: 6

In [25]:
idxrev = {var.item():key for key,var in idxs.items()}
xsoldict = {str(idxrev[key.item()]): xsol.x[idx] for idx,key in enumerate(solidxs)}
xidxed = load_vals(xsoldict, indices, isdict=True, default=1.1)
xsolvalall = f_MDF.analysis(xidxed)
print_formatted_table([xsolvalall], idxs, idxrev)

b      d    hkm EN     m_A   m_s   r      T        D        l_c   Hval   P_T   Lp    D_T  m_b   a      A     g    h      m_t   D_p   X_r m_pr  E_b    P_c    Ln    m_p   m_T   Lt rhoval   G_r     
2.31e7 0.61 400 14.125 0.509 1.161 2.29e6 5553.459 1.28e+11 0.136 5.82e4 0.725 9.323 0.39 0.232 6.78e6 0.051 0.11 4.00e5 5.804 0.049 5   3.833 1.16e5 12.725 0.677 0.022 0.049 10 2.72e-12 8255.002


In [16]:
f_MDF.residual(xsolvalall.clone())

tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00, 1.1102e-16, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 2.2204e-16, 0.0000e+00, 0.0000e+00, 0.0000e+00,
        0.0000e+00, 0.0000e+00, 0.0000e+00], dtype=torch.float64)

### AAO/IDF

In [91]:
x0 = {"hkm": 400, "m_pr": 1, "X_r": 5, "A":0.05}
xidxed = load_vals(x0, indices, isdict=True, default=1.1)
xguess2 = f_MDF.analysis(xidxed)
sets,_,_,_, indices = SPF_AAO.gather_sets()

In [92]:
obj, dobj, xguess, cons, idxs, solidxs = SPF_IDF.build_opt(x0={str(key): xguess2[val].item() for key,val in indices.items()})

In [93]:
xsol = optimize.minimize(obj, xguess, jac=dobj, 
                         constraints=cons, method='SLSQP')

In [86]:
obj, dobj, xguess, cons, idxs, solidxs = SPF_AAO.build_opt(x0={str(key): xguess3[val].item() for key,val in indices.items()})

In [77]:
xsol = optimize.minimize(obj, xguess, jac=dobj, 
                         constraints=cons, method='SLSQP')

In [94]:
xsol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 5.804200911102231
       x: [ 5.804e+00  5.000e+00 ...  9.323e+00  1.161e+00]
     nit: 15
     jac: [ 1.000e+00  0.000e+00 ...  0.000e+00  0.000e+00]
    nfev: 27
    njev: 15

In [79]:
idxrev = {var.item():key for key,var in idxs.items()}
xsolval = xguess2.clone().numpy()
xsolval[solidxs] = xsol.x
print_formatted_table([xguess2, xsolval], idxs, idxrev)

b      d    hkm     EN     m_A   m_s   r      T        D        l_c   Hval   P_T   Lp    D_T  m_b   a      A     g    h      m_t   D_p   X_r m_pr  E_b    P_c    Ln    m_p   m_T   Lt    rhoval   G_r     
2.31e7 0.61 400     9.861  0.5   0.22  2.29e6 5553.459 1.28e+11 0.136 5.82e4 0.506 2.475 0.39 0.228 6.78e6 0.05  0.11 4.00e5 2.018 0.049 5   1     1.14e5 12.506 0.131 0.022 0.049 2.606 2.72e-12 8255.002
2.31e7 0.61 399.999 14.125 0.509 1.161 2.29e6 5553.458 1.28e+11 0.136 5.82e4 0.725 9.323 0.39 0.232 6.78e6 0.051 0.11 4.00e5 5.804 0.049 5   3.833 1.16e5 12.725 0.677 0.022 0.049 10    2.72e-12 8255.002


### Reconfiguring architectures

In [ ]:
SPF_IDF = FPF.config(parallel=MDA.supersets) # should eliminate state variables
SPF_MDF_full = FPF.config(elim=[MDA.config(residuals=MDA.supersets)])
scc_order = [{2}, {1}, {0}]
SPF_MDF_compact = FPF.config_from_order(scc_order)
outset_tear, wf_tear = None, None
SPF_MDF_tear = FPF.reconfigure(outset_tear).config_from_workflow(wf_tear)
outset_tear_extended, wf_tear_extended = None, None
SPF_MDF_tear_extended = FPF.reconfigure(outset_tear_extended).config

In [7]:
MDA1 = MFunctionalSet(Orbit, Propulsion).config(elim=[Orbit, Propulsion])
sets,_,_,_, indices = MDA1.gather_sets()
f = MDA1.build()

In [10]:
x0 = {"h": 400e3, "a": 400e3 + 6371e3, "m_t": 1.5}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

d    Lp    rhoval r      a      Hval   T        Ln    h      m_t Lt    g   
0.61 1.236 0      2.29e6 6.77e6 5.82e4 5544.858 0.178 4.00e5 1.5 1.414 0.11


In [149]:
MDAp = MFunctionalSet(Orbit, Power, Comms).config(elim=[Orbit, Power, Comms])
_,_,_,_, indices = MDAp.gather_sets()
f = MDAp.build()
x0 = {"h": 400e3, "D": 0.1}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

d    EN       r      P_T   a      h      m_A m_T   T        D_T   D   g    G_r      b        E_b    m_b   P_c    l_c  
0.61 4.51e+12 2.29e6 0.489 6.77e6 4.00e5 0.5 0.023 5544.858 0.237 0.1 0.11 8255.002 1.80e-05 1.14e5 0.227 12.489 0.136


In [150]:
_,_,_,_, indices = Power.gather_sets()
f = Power.build()
x0 = {"d": 0.6, "T": 6000}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

T    E_b    d   m_b   P_c    P_T   m_A
6000 1.23e5 0.6 0.246 12.285 0.285 0.5


In [151]:
_,_,_,_, indices = Orbit.gather_sets()
f = Orbit.build()
x0 = {"h": 400e3}
xidxed = load_vals(x0, indices, isdict=True)
f.analysis(xidxed)
idxrev = {var.item():key for key,var in indices.items()}
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

r      T        d    a      h      g   
2.29e6 5544.858 0.61 6.77e6 4.00e5 0.11


In [200]:
SPF_MDF1 = FPF.config(elim=[HMult, Orbit, Power, Payload, Comms])
sets, ineq_constraints_merged, eq_constraints, obj, indices = SPF_MDF1.gather_sets()
f = SPF_MDF1.build()
x0 = {"hkm": 400, "X_r": 1}
xidxed = load_vals(x0, indices, isdict=True)
f0 = f.analysis(xidxed).detach().clone()
idxrev = {var.item():key for key,var in indices.items()}
valdict = {val:f0[key].item() for key,val in idxrev.items()}
# print_formatted_table([f.analysis(xidxed)], indices, idxrev)

In [202]:
obj_function, dobj, xguess, constraints  = SPF_MDF1.build_opt(x0=f0)

In [203]:
dobj(xguess), constraints[0]['fun'](xguess)

(array([ 0.00095426, -0.36158133]),
 tensor([-4.0000e+00,  9.9000e+01,  1.0000e+05], dtype=torch.float64))

In [208]:
xsol = optimize.minimize(obj_function, xguess, jac=dobj, constraints=constraints, method='SLSQP', 
                         options={'ftol': 1e-9})

In [209]:
xsol.x

array([300., 100.])

In [211]:
xidxed = load_vals({'hkm':xsol.x[0], 'X_r':xsol.x[1]}, 
                   indices, isdict=True)
print_formatted_table([f.analysis(xidxed)], indices, idxrev)

d     EN     r      P_T   a      h      m_A m_T   X_r T        D_p      D_T   D      g     G_r      hkm b      E_b    m_b   P_c    l_c   m_p     
0.596 37.757 1.98e6 0.199 6.67e6 3.00e5 0.5 0.023 100 5422.476 1.83e-03 0.237 6.40e9 0.096 8255.002 300 1.18e6 1.11e5 0.222 12.199 0.136 1.57e-04


In [212]:
xsol

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 0.7452754334567794
       x: [ 3.000e+02  1.000e+02]
     nit: 27
     jac: [ 5.071e-05 -2.349e-06]
    nfev: 27
    njev: 27

In [77]:
print_formatted_table([f0], indices, idxrev)

T        X_r      d    D_p   D        g    r      E_b    P_T   m_b   h      P_c    m_p   a      m_A
5544.858 2085.041 0.61 0.244 6.40e+11 0.11 2.29e6 1.14e5 0.489 0.227 4.00e5 12.489 0.241 6.77e6 0.5


In [ ]:
SPF_IDF = FPF.config(parallel=[Orbit, Power, Payload])
SPF_AAO = FPF.config(residuals=Orbit.supersets+Power.supersets+Payload.supersets)
SPF_MDF1 = FPF.config(elim=[Orbit, Power, Payload])
SPF_MDF2 = FPF.config(elim=[MDA.config(parallel=[Orbit, Power, Payload])])